In [64]:
# gerekli kütüphaneler
import tensorflow.keras  
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import glob
import os
from PIL import Image

In [40]:
# uyarı mesajlaırnı engelleme
import warnings
warnings.filterwarnings("ignore")

In [41]:
# resim boytu

widht = 125 
height = 50

In [50]:
# resimleri içe aktarma
imgs = glob.glob("./img/*.png")

# resimler
x = []
# labels - etiketler
y = []

for img in imgs:
  fileName = os.path.basename(img)
  label = fileName.split("_")[0]
  # resize - yeniden boyutlandırma
  resim = np.array(Image.open(img).convert("L").resize((widht, height)))
  # scaling 
  resim = resim / 255

  # listeye ekleme
  x.append(resim)
  y.append(label)
print(len(x))
print(len(y))

300
300


In [51]:
x = np.array(x)
x = x.reshape(x.shape[0], widht, height, 1) # siyah beyaz olarak kaydetme

ONEHOT ENCODİNG İŞLEMİ

One Hot Encoding, kategorik değişkenlerin ikili (binary) olarak temsil edilmesi anlamına gelmektedir. Bu işlem ilk önce kategorik değerlerin tamsayı değerleriyle eşlenmesini gerektirir. Daha sonra, her bir tamsayı değeri, 1 ile işaretlenmiş tamsayı indeksi dışında ki tüm değerleri   sıfır  olan bir ikili vektör olarak temsil edilir.

In [52]:
# etiketleri sayısal ifadelere çevirme
print(y)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)
integer_encoded = integer_encoded.reshape((len(integer_encoded),1))
# one hot encoding 
onehot_encoder = OneHotEncoder(sparse = False) # sparse matris elde etmek istemediğimiz için false
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

y = onehot_encoded

['down', 'down', 'right', 'up', 'right', 'down', 'right', 'right', 'up', 'down', 'right', 'up', 'up', 'down', 'up', 'right', 'down', 'right', 'right', 'right', 'right', 'down', 'up', 'up', 'down', 'right', 'right', 'up', 'down', 'down', 'down', 'down', 'down', 'up', 'right', 'up', 'down', 'up', 'up', 'down', 'down', 'right', 'down', 'up', 'up', 'right', 'up', 'right', 'right', 'up', 'down', 'down', 'up', 'right', 'right', 'down', 'right', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'right', 'right', 'right', 'right', 'up', 'up', 'up', 'right', 'up', 'down', 'up', 'right', 'up', 'down', 'right', 'down', 'up', 'up', 'right', 'down', 'down', 'down', 'right', 'up', 'right', 'down', 'down', 'up', 'up', 'down', 'up', 'right', 'right', 'down', 'down', 'up', 'up', 'right', 'right', 'up', 'up', 'down', 'right', 'up', 'up', 'up', 'down', 'right', 'right', 'down', 'down', 'up', 'up', 'down', 'right', 'down', 'right', 'right', 'up', 'down', 'down', 'up', 'down', 'down', 'down', 'down', 'right'

Veri setini test ve train olarak ayırma

In [54]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state=2)

Model Oluşturma

In [66]:
from keras.backend import conv2d
model = Sequential()
model.add(Conv2D(32, kernel_size= (3,3), activation = "relu", input_shape=(widht, height, 1 )))

model.add(Conv2D(64, kernel_size = (3,3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(3, activation="softmax"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 123, 48, 32)       320       
                                                                 
 conv2d_3 (Conv2D)           (None, 121, 46, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 60, 23, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 60, 23, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 88320)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               11305088  
                                                      

Model Compiling

In [67]:
model.compile(loss= "categorical_crossentropy",
              optimizer = "Adam",
              metrics= ["accuracy"])

Model Eğitimi

In [70]:
model.fit(x_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
4/4 [==============================] - 3s 199ms/step - loss: 4.8430 - accuracy: 0.2933
Epoch 2/50
4/4 [==============================] - 0s 65ms/step - loss: 1.5294 - accuracy: 0.4311
Epoch 3/50
4/4 [==============================] - 0s 66ms/step - loss: 0.9715 - accuracy: 0.5422
Epoch 4/50
4/4 [==============================] - 0s 66ms/step - loss: 0.7420 - accuracy: 0.7067
Epoch 5/50
4/4 [==============================] - 0s 67ms/step - loss: 0.6346 - accuracy: 0.7556
Epoch 6/50
4/4 [==============================] - 0s 67ms/step - loss: 0.4878 - accuracy: 0.8578
Epoch 7/50
4/4 [==============================] - 0s 67ms/step - loss: 0.4141 - accuracy: 0.8178
Epoch 8/50
4/4 [==============================] - 0s 67ms/step - loss: 0.3511 - accuracy: 0.8711
Epoch 9/50
4/4 [==============================] - 0s 67ms/step - loss: 0.2944 - accuracy: 0.8889
Epoch 10/50
4/4 [==============================] - 0s 64ms/step - loss: 0.2627 - accuracy: 0.9156
Epoch 11/50
4/4 [===========

Model Doğruluk oranı

In [72]:
# train verisi üzerindeki doğruluk oranı
model.evaluate(x_train, y_train)

# test verisi ğzerindeki doğruluk oranı
model.evaluate(x_test,  y_test)

3/3 [==============================] - 0s 77ms/step - loss: 0.5541 - accuracy: 0.8933


[0.5540509819984436, 0.8933333158493042]

Oluşturduğumuz Modeli Kaydetme

In [73]:
open("model.json", "w").write(model.to_json())
model.save_weights("trex_weight.h5")